Import module

In [ ]:
#Import talib
!wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
!tar -xzvf ta-lib-0.4.0-src.tar.gz
%cd ta-lib
!./configure --prefix=/usr
!make
!make install
!pip install Ta-Lib
import talib
from talib import abstract

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
from datetime import datetime
import pandas_datareader as pdr
import statistics
import math
from plotly.offline import plot, iplot, init_notebook_mode
import plotly.graph_objs as go
from dataclasses import dataclass

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import gspread
from google.colab import drive
from google.auth import default
creds, _ = default()
auth.authenticate_user()
gc = gspread.authorize(creds)
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

Backtest function

In [ ]:
# Deposit Function


def deposit_func(trade_record, date, price, month_add):
    fund = trade_record.iloc[-1, -3] + month_add
    storage = trade_record.iloc[-1, -4]
    trade_record_1 = record_func(
        trade_record=trade_record, date=date, change_types=5, price=price, storage=storage, fund=fund)
    return trade_record_1



# Buy Function, input price and fund

def buy_func(trade_record, date, price):
    fund = trade_record.iloc[-1, -3]
    storage = trade_record.iloc[-1, -4]
    stock_volume_thousand = math.floor(fund/(price*1000))
    transaction_amount = (stock_volume_thousand * 1000) * price
    commission_fee = math.floor(transaction_amount * trade_class.commission)
    fund = fund-(transaction_amount)-commission_fee
    storage = storage + stock_volume_thousand * 1000
    trade_record_1 = record_func(trade_record, date, change_types=1, stock_volume=stock_volume_thousand * 1000,
                                 transaction_amount=transaction_amount, price=price, storage=storage, fund=fund, transaction_cost=commission_fee)
    return trade_record_1


# Sell Function, input price, storage, fund
def sell_func(trade_record, date, price, storage, fund):
    fund = trade_record.iloc[-1, -3]
    storage = trade_record.iloc[len(trade_record, -4)]
    stock_volume_thousand = math.floor(storage/1000)
    transaction_amount = (stock_volume_thousand*price)
    commission_fee = math.floor(
        transaction_amount * (trade_class.commission + trade_class.tax))
    fund = math.floor(fund + transaction_amount - commission_fee)
    storage = storage - stock_volume_thousand*1000
    trade_record_1 = record_func(trade_record, date, change_types=2, stock_volume=stock_volume_thousand*1000, transaction_amount=transaction_amount,
                                 price=price, storage=storage, fund=fund, transaction_cost=commission_fee)
    return trade_record_1


# Cash Dividend Function, input cash dividend, storage
def cash_dividend_func(trade_record, date, cash_dividend, price, storage):
    fund = trade_record.iloc[-1, -3]
    storage_1 = trade_record.iloc[-1, -4]
    cash_dividend_1 = math.floor(cash_dividend * storage)
    fund = fund + cash_dividend_1
    trade_record_1 = record_func(trade_record, date, change_types=3, price=price, storage=storage_1, fund=fund, transaction_amount=cash_dividend_1)
    return trade_record_1


# Stock Dividend Function, input stock dividend, storage
def stock_dividend_func(trade_record, date, stock_dividend, price, storage):
    fund = trade_record.iloc[-1, -3]
    stock_dividend_1 = math.floor((stock_dividend/100) * storage)
    storage = storage + stock_dividend_1
    trade_record_1 = record_func(trade_record=trade_record, date=date, change_types=4,
                                 price=price, storage=storage, fund=fund, stock_volume=stock_dividend_1)
    return trade_record_1




def record_func(trade_record, date, change_types, stock_volume=0, transaction_amount=0, price=0, storage=0, fund=0, transaction_cost=0):  # Record Function
    asset = price*storage+fund
    record_list = [date, change_types,
                   stock_volume, transaction_amount, price, storage, fund, asset, transaction_cost]
    trade_record.loc[len(trade_record)] = record_list
    return trade_record

@dataclass
class trading:
  fund: float = 20000
  fund_month_added: int = 5000
  commission: float = 0.001425
  tax: float = 0.003
  cash_dividend: float = 0
  stock_dividend: float = 0
  cash_dividend_stoarge: int = 0
  stock_dividend_stoarge: int = 0

trade_class = trading()

Import data

In [ ]:
#Import price data
stock_price = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/', sheet_name='')
#Keep the column which will be used
stock_price_1 = stock_price[['年月日','開盤價(元)','最高價(元)','最低價(元)','收盤價(元)','成交量(千股)']]

In [ ]:
#Cash dividend schedual
cash_dividend = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Stock/', sheet_name='')
#Mark the cash dividend payment day
cash_dividend['股息發放'] = cash_dividend.index
#Rename column
cash_dividend = cash_dividend.rename(columns={'息值(元)':'息值'})
#Keep the column which will be used
cash_dividend_1 = cash_dividend[['除息日','息值']]
cash_dividend_2 = cash_dividend[['股息發放日','股息發放']]

In [ ]:
#Stock dividend schedual
stock_dividend = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Stock/', sheet_name='')
#Mark the stock dividend payment day
stock_dividend['股利上市'] = stock_dividend.index
#Rename column
stock_dividend = stock_dividend.rename(columns={'無償配股％':'無償配股'})
#Keep the column which will be used
stock_dividend_1 = stock_dividend[['除權日','無償配股']]
stock_dividend_2 = stock_dividend[['股票股利上市日','股利上市']]

Merge data

In [ ]:
stock_price_1 = stock_price_1.merge(cash_dividend_1, how='outer',left_on='年月日',right_on='除息日')
stock_price_1 = stock_price_1.merge(cash_dividend_2, how='outer',left_on='年月日',right_on='股息發放日')
#Move to one trading day before
stock_price_1['除息日'] = stock_price_1.除息日.shift(-1)
stock_price_1['息值'] = stock_price_1.息值.shift(-1)

In [ ]:
stock_price_1 = stock_price_1.merge(stock_dividend_1, how='outer',left_on='年月日',right_on='除權日')
stock_price_1 = stock_price_1.merge(stock_dividend_2, how='outer',left_on='年月日',right_on='股票股利上市日')
#Move to one trading day before
stock_price_1['除權日'] = stock_price_1.除權日.shift(-1)
stock_price_1['無償配股'] = stock_price_1.無償配股.shift(-1)

Backtest

In [ ]:
#Mark deposit date
stock_price_1['Day'] = stock_price_1['年月日'].dt.day
stock_price_1['Deposit'] = np.where(stock_price_1['Day']>stock_price_1['Day'].shift(-1),1,0)

In [ ]:
# Setting record table and transcost
column1 = ['Date', 'Change types', 'Stock Volume', 'Transaction Amount', 'Price',
           'Storage', 'Fund', 'Asset', 'Transaction Cost']
trade_record_2 = pd.DataFrame(columns=column1)
record_list_1 = [0, 0, 0, 0, 0, 0, 10000, 0, 0]
trade_record_2.loc[0] = record_list_1

In [ ]:
for i in range(0, len(stock_price_1)):
    #Deposit
    if stock_price_1.loc[i,'Deposit']==1:
      trade_record_2 = deposit_func(
          trade_record = trade_record_2,
          date = stock_price_1.loc[i, '年月日'],
          price = stock_price_1.loc[i, '收盤價(元)'],
          month_add = trade_class.fund_month_added
          )
    #Cash Dividend
    if stock_price_1.loc[i,'息值']>0:
      trade_class.cash_dividend = stock_price_1.loc[i,'息值']
      trade_class.cash_dividend_stoarge = trade_record_2.iloc[-1, -4]
    if stock_price_1.loc[i,'股息發放']>-1:
      trade_record_2 = cash_dividend_func(
          trade_record =  trade_record_2,
          date = stock_price_1.loc[i,'年月日'],
          cash_dividend = trade_class.cash_dividend,
          price = stock_price_1.loc[i,'收盤價(元)'],
          storage = trade_class.cash_dividend_stoarge
          )
      trade_class.cash_dividend = 0
      trade_class.cash_dividend_stoarge = 0
    #Stock Dividend
    if stock_price_1.loc[i,'無償配股']>0:
       trade_class.stock_dividend = stock_price_1.loc[i,'無償配股']
       trade_class.stock_dividend_stoarge = trade_record_2.iloc[-1, -4]
    if stock_price_1.loc[i,'股利上市']>-1:
       trade_record_2 = stock_dividend_func(
           trade_record = trade_record_2,
           date = stock_price_1.loc[i,'年月日'],
           stock_dividend = trade_class.stock_dividend,
           price = stock_price_1.loc[i,'收盤價(元)'],
           storage = trade_class.stock_dividend_stoarge
       )
       trade_class.stock_dividend = 0
       trade_class.stock_dividend_stoarge = 0
    #Buy
    if (trade_record_2.iloc[-1,-3]/(stock_price_1.loc[i,'收盤價(元)']*1000)) >=1:
      trade_record_2 = buy_func(
          trade_record =  trade_record_2,
          date = stock_price_1.loc[i,'年月日'],
          price = stock_price_1.loc[i,'收盤價(元)']
          )

In [ ]:
trade_record_2.head()